# *** LibraryBot - Sistema de Biblioteca Digital (Modo Interactivo) ***
## Chatbot bibliotecario que usa OpenLibrary como catálogo

In [14]:
!pip install openai  #Instalación de librería open Ai

In [15]:
from openai import OpenAI
import getpass
import requests
import random


### Configuración open Ai

In [16]:
print("🔧 Configurando LibraryBot...")
api_key = getpass.getpass("Ingresa tu clave API de OpenAI: ")
client = OpenAI(api_key = api_key)

def get_completion_from_messages(messages, model="gpt-4.1-mini", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content

🔧 Configurando LibraryBot...
Ingresa tu clave API de OpenAI: ··········


### Integración con OpenLibrary

In [17]:
def buscar_en_catalogo_biblioteca(consulta, limite=10):
    try:
        print(f"📚 Consultando catálogo: {consulta}")
        url = "https://openlibrary.org/search.json"
        params = {
            'q': consulta,
            'limit': limite,
            'fields': 'key,title,author_name,first_publish_year,subject,language'
        }
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        datos = response.json()

        libros_biblioteca = []
        for libro in datos.get('docs', []):
            libro_biblioteca = procesar_libro_para_biblioteca(libro)
            libros_biblioteca.append(libro_biblioteca)

        return {"libros": libros_biblioteca, "total": len(libros_biblioteca)}
    except Exception as e:
        print(f"❌ Error: {e}")
        return {"libros": [], "total": 0}



In [18]:
def procesar_libro_para_biblioteca(datos_libro):
    titulo = datos_libro.get('title', 'Título desconocido')
    autores = datos_libro.get('author_name', ['Autor desconocido'])
    año = datos_libro.get('first_publish_year', 'Año desconocido')
    temas = datos_libro.get('subject', [])

    # Simular disponibilidad
    total_copias = random.randint(1, 8)
    copias_prestadas = random.randint(0, total_copias)
    copias_disponibles = total_copias - copias_prestadas

    # Período de préstamo según categoría
    periodo_prestamo = determinar_periodo_prestamo(temas)

    # Fecha disponible si prestado
    fecha_disponible = None
    if copias_disponibles == 0:
        dias = random.randint(3, 21)
        fecha_disponible = f"{dias} días"

    idiomas = datos_libro.get('language', [])
    tiene_español = any('spa' in str(idioma).lower() or 'spanish' in str(idioma).lower() for idioma in idiomas)

    return {
        'titulo': titulo,
        'autores': ', '.join(autores) if isinstance(autores, list) else str(autores),
        'año': año,
        'temas': temas[:3] if temas else [],
        'total_copias': total_copias,
        'copias_disponibles': copias_disponibles,
        'periodo_prestamo': periodo_prestamo,
        'fecha_disponible': fecha_disponible,
        'tiene_español': tiene_español
    }

   ### Verificar español

In [19]:
def determinar_periodo_prestamo(temas):
    temas_str = ' '.join(temas).lower() if temas else ''

    if any(palabra in temas_str for palabra in ['programming', 'computer', 'science', 'mathematics', 'engineering']):
        return '21 días'
    elif any(palabra in temas_str for palabra in ['reference', 'dictionary', 'encyclopedia']):
        return '7 días'
    else:
        return '14 días'

def formatear_libro_biblioteca(libro):
    info = f'📖 "{libro["titulo"]}" por {libro["autores"]} ({libro["año"]})'

    if libro['copias_disponibles'] > 0:
        info += f' - ✅ {libro["copias_disponibles"]}/{libro["total_copias"]} disponibles'
        info += f' - Préstamo: {libro["periodo_prestamo"]}'
    else:
        info += f' - ❌ No disponible ({libro["total_copias"]} prestadas)'
        if libro['fecha_disponible']:
            info += f' - Disponible en: {libro["fecha_disponible"]}'

    if libro['tiene_español']:
        info += ' - 🇪🇸 En español'

    if libro['temas']:
        info += f' - Temas: {", ".join(libro["temas"])}'

    return info

def procesar_busqueda_biblioteca(consulta, limite=5):
    resultados = buscar_en_catalogo_biblioteca(consulta, limite)

    if resultados['total'] == 0:
        return "No encontré libros. ¿Podrías intentar otros términos?"

    texto = f"Encontré {resultados['total']} libros en nuestro catálogo:\n\n"
    for i, libro in enumerate(resultados['libros'], 1):
        texto += f"{i}. {formatear_libro_biblioteca(libro)}\n\n"

    texto += "¿Te gustaría reservar alguno o buscar algo más específico?"
    return texto

### Contexto del chatbot

In [20]:
contexto_biblioteca = [{'role':'system', 'content':"""
Eres LibraryBot, asistente de nuestra Biblioteca Digital.

FUNCIONES:
- Ayudar a buscar libros en el catálogo
- Gestionar reservas y préstamos
- Verificar disponibilidad y períodos
- Sugerir alternativas cuando no están disponibles
- Procesar reservas con resúmenes finales

SISTEMA:
- Períodos: 7, 14 o 21 días según categoría
- Si está prestado, informa cuándo estará disponible
- Puedes reservar para recoger o entrega

INSTRUCCIONES:
1. Saluda como bibliotecario profesional
2. Presenta resultados con disponibilidad clara
3. Para no disponibles, sugiere alternativas
4. Recopila reservas y confirma detalles
5. Al final, crea resumen con fechas
6. Cuando terminen ("gracias", "eso es todo"), TERMINA educadamente
7. Usa temperatura 0.3 SOLO para sugerencias creativas

Responde en español, profesional pero amigable.
"""}]


### Modo interactivo

In [21]:
def biblioteca_interactiva():
    mensajes = contexto_biblioteca.copy()
    saludo = '¡Bienvenido a nuestra Biblioteca Digital! Soy tu asistente bibliotecario. Puedo ayudarte a buscar libros, verificar disponibilidad y gestionar préstamos. ¿En qué puedo ayudarte hoy?'
    mensajes.append({'role':'assistant', 'content': saludo})

    print("📚 Bibliotecario:", saludo)

    palabras_busqueda = ['buscar', 'busco', 'libro', 'libros', 'autor', 'tema', 'reservar', 'prestamo', 'disponible']
    finalizadores = ["gracias por usar", "que tengas un buen día", "disfruta la lectura", "hasta la próxima"]

    while True:
        entrada_usuario = input("\n👤 Usuario: ")

        if entrada_usuario.lower() in ['salir', 'adiós', 'gracias', 'exit']:
            print("📚 Bibliotecario: ¡Gracias por visitar nuestra biblioteca! ¡Disfruta tus libros!")
            break

        mensajes.append({'role':'user', 'content': entrada_usuario})

        # Detectar si necesita búsqueda
        if any(palabra in entrada_usuario.lower() for palabra in palabras_busqueda):
            print("📚 Consultando catálogo...")
            resultados = procesar_busqueda_biblioteca(entrada_usuario, 5)
            prompt_sistema = f"Consulta: {entrada_usuario}\n\nResultados:\n{resultados}\n\nResponde como bibliotecario."
            mensajes.append({'role':'system', 'content': prompt_sistema})

            respuesta = get_completion_from_messages(mensajes, temperature=0.2)
        else:
            respuesta = get_completion_from_messages(mensajes, temperature=0)

        mensajes.append({'role':'assistant', 'content': respuesta})
        print("📚 Bibliotecario:", respuesta)

        # Verificar fin de conversación
        if any(fin in respuesta.lower() for fin in finalizadores):
            print("\n--- Sesión terminada ---")
            break

### Programa principal

In [22]:
if __name__ == "__main__":
    print("📚 LibraryBot - Biblioteca Digital Interactiva")
    print("=" * 45)
    print("Sistema bibliotecario con catálogo OpenLibrary")
    print("Escribe 'salir' para terminar")
    print("=" * 45)
    biblioteca_interactiva()

📚 LibraryBot - Biblioteca Digital Interactiva
Sistema bibliotecario con catálogo OpenLibrary
Escribe 'salir' para terminar
📚 Bibliotecario: ¡Bienvenido a nuestra Biblioteca Digital! Soy tu asistente bibliotecario. Puedo ayudarte a buscar libros, verificar disponibilidad y gestionar préstamos. ¿En qué puedo ayudarte hoy?

👤 Usuario: Tienes la Muerte Roja de Edgard Alan Poe?
📚 Bibliotecario: He buscado en nuestro catálogo y no aparece un título exacto llamado "La Muerte Roja" de Edgar Allan Poe. Sin embargo, es posible que te refieras al cuento "La máscara de la muerte roja" ("The Masque of the Red Death"), que es una obra muy conocida de Poe.

¿Quieres que verifique la disponibilidad de "La máscara de la muerte roja" o prefieres que te sugiera otros cuentos o colecciones de Edgar Allan Poe disponibles en nuestra biblioteca?

👤 Usuario: Sí 
📚 Bibliotecario: Perfecto, verifico la disponibilidad de "La máscara de la muerte roja" de Edgar Allan Poe.

— El cuento está disponible en una antol

KeyboardInterrupt: Interrupted by user